In [1]:
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import csv
import re
from pandas.core.frame import DataFrame
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument('--no-sandbox')

In [4]:
#店面地址轉換GPS
def get_coordinate(addr):
    browser = webdriver.Chrome(executable_path='chromedriver',options=options)
    browser.get("http://www.map.com.tw/")
    search = browser.find_element_by_id("searchWord")
    search.clear()
    search.send_keys(addr)
    browser.find_element_by_xpath("/html/body/form/div[10]/div[2]/img[2]").click() 
    time.sleep(2)
    iframe = browser.find_elements_by_tag_name("iframe")[1]
    browser.switch_to.frame(iframe)
    coor_btn = browser.find_element_by_xpath("/html/body/form/div[4]/table/tbody/tr[3]/td/table/tbody/tr/td[2]")
    coor_btn.click()
    coor = browser.find_element_by_xpath("/html/body/form/div[5]/table/tbody/tr[2]/td")
    coor = coor.text.strip().split(" ")
    lat = coor[-1].split("：")[-1]
    log = coor[0].split("：")[-1]
    browser.quit()
    return (lat, log)

In [7]:
final = pd.DataFrame()

city=['基隆市','台北市','新北市','宜蘭縣','新竹市','新竹縣','桃園市','苗栗縣','台中市','彰化縣','南投縣','嘉義市','嘉義縣',
      '雲林縣','台南市','高雄市','屏東縣','台東縣','花蓮縣','金門縣','連江縣','澎湖縣']

for i in city:
    post_params = {
        'data[name]': '',
        'data[county]': i,
        'data[district]': '',
        'data[zipcode]': '',
        'data[address]':'',
    }

    #擷取店面店名
    response = requests.post('https://www.louisacoffee.co/visit_result', data = post_params)
    #response.text
    
    pattern='<h4>.+</h4>'
    d=re.compile(pattern=pattern).findall(response.text)
    d

    name=[]
    for i in d:
        i = i.strip('</h4>')
    #     print(i)
        name.append(i)
    
    #擷取店面地址
    pattern='rel-store-address=\S+\s+\S+\s+\S+'
    d=re.compile(pattern=pattern).findall(response.text)
    a=d[::2]
    
    add=[]
    for i in a:
        i = i.strip('rel-store-address="')
    #     print(i)
        add.append(i)

    #擷取店面營業時間
#     pattern='rel-store-date=\S+ [0-9]+:[0-9]+-[0-9]+:[0-9]+'
#     d=re.compile(pattern=pattern).findall(response.text)
#     a=d[::2]

#     hours=[]
#     for i in a:
#         i = i.strip('rel-store-date="')
#     #     print(i)
#         hours.append(i)
#     hours
    
    #擷取店面電話
#     pattern='電話/[0-9][0-9]-[0-9][0-9][0-9][0-9]-[0-9][0-9][0-9][0-9]'
#     d=re.compile(pattern=pattern).findall(response.text)
#     d
#     tel=[]
#     for i in d:
#         i = i.strip('電話/')
#     #     print(i)
#         tel.append(i)
    
    #獲取店面GPS
    storeGps=[]
    for i in add:
        print(i)
        a=get_coordinate(i)
        print(a)
        storeGps.append(a)
    storeGps

    #from pandas.core.frame import DataFrame
    c={"storeName" : name,
       "storeAdd" : add,
#        "hours" : hours,
#        "tel" : tel,
       "storeGps":storeGps,
       }
    
    #將爬取資料轉換成DataFrame
    data=DataFrame(c)
    
    #每個縣市資料添加上去
    final=final.append(data,ignore_index=True) 

final

基隆市 仁愛區 孝三路109號
('25.131168', '121.739754')
基隆市 中正區 信一路168號1樓
('25.130381', '121.745334')
基隆市 安樂區 武嶺街136號
('25.140018', '121.710334')
基隆市 仁愛區 仁二路66號
('25.127855', '121.749247')
基隆市 中正區 新豐街233號
('25.136753', '121.78828')
基隆市 安樂區 麥金路231之5號
('25.121775', '121.721052')
基隆市 七堵區 福五街28號
('25.083173', '121.699217')
台北市 松山區 民生東路四段56巷3弄1號
('25.057214', '121.551737')
台北市 大安區 敦化南路二段81巷45弄21號
('25.030518', '121.550353')
台北市 中正區 紹興南街4-1號
('25.043636', '121.524736')
台北市 信義區 逸仙路50巷6號
('25.040598', '121.56209')
台北市 中正區 衡陽路112號
('25.042149', '121.509925')
台北市 中正區 延平南路214號
('25.034531', '121.50738')
台北市 中正區 杭州南路一段153號
('25.035868', '121.524635')
台北市 中正區 重慶南路一段77-2號
('25.044044', '121.51352')
台北市 中正區 臨沂街27巷9-4號
('25.04127', '121.531399')
台北市 中正區 新生南路一段130號
('25.036931', '121.532484')
台北市 大同區 民生西路132號
('25.057351', '121.519025')
台北市 大同區 酒泉街139號
('25.072165', '121.514291')
台北市 中山區 四平街55號
('25.053758', '121.533502')
台北市 中山區 松江路313巷22號之2
('25.061692', '121.534718')
台北市 中山區 中山北路一段140巷138號(後棟)
('25.05133', '121

('25.000381', '121.487868')
新北市 中和區 莒光路158號
('25.000525', '121.472519')
新北市 中和區 員山路492號
('25.007095', '121.479858')
新北市 中和區 景安路175-2號
('24.994177', '121.505255')
新北市 中和區 自立路5號
('24.996604', '121.522573')
新北市 永和區 永和路二段92巷3號1樓
('25.011546', '121.515244')
新北市 永和區 永元路95巷3號
('25.001986', '121.522276')
新北市 永和區 竹林路160號
('25.01043', '121.520261')
新北市 永和區 中山路一段271號
('25.007207', '121.507257')
新北市 永和區 得和路160號
('25.000062', '121.519537')
新北市 汐止區 新北市汐止區新台五路一段75號B1
('25.060916', '121.645726')
新北市 汐止區 新台五路一段176號
('25.062527', '121.654691')
新北市 汐止區 建成路60號
('25.073157', '121.663394')
新北市 汐止區 中興路226號
('25.066694', '121.63083')
新北市 板橋區 文化路一段17號
('25.010233', '121.45761')
新北市 板橋區 文化路一段309之37號1樓
('25.020288', '121.465631')
新北市 板橋區 新民街43號1樓
('25.010809', '121.463887')
新北市 板橋區 文化路二段367號
('25.031055', '121.473078')
新北市 板橋區 南雅南路二段130號
('24.996805', '121.451901')
新北市 板橋區 館前東路9號
('25.007272', '121.45983')
新北市 板橋區 重慶路195號
('25.001993', '121.462904')
新北市 板橋區 雨農路58號
('25.024204', '121.460686')
新北市 板橋區 文化路二段113號
('

('24.996149', '121.294943')
桃園市 大溪區 慈湖路22號
('24.883809', '121.289101')
苗栗縣 頭份市 和平路107號
('24.685743', '120.906061')
苗栗縣 竹南鎮 大營路56號
('24.682735', '120.873236')
苗栗縣 頭份市 建國路107號
('24.694661', '120.911491')
苗栗縣 苗栗市 建功里民族路70-1與70-2號
('24.563325', '120.818916')
台中市 西屯區 福康路26號
('24.183616', '120.617011')
台中市 西屯區 河南路2段301巷72號
('24.171923', '120.648295')
台中市 北屯區 北平路三段146號
('24.170848', '120.683916')
台中市 南屯區 大墩路806號
('24.154616', '120.649945')
台中市 大甲區 育德路21號
('24.345869', '120.622575')
台中市 西屯區 朝富路286號
('24.162206', '120.634834')
台中市 北區 進化路571-2號
('24.159443', '120.694187')
台中市 西屯區 文心路二段560-9號
('24.156756', '120.647076')
台中市 南屯區 文心路一段376號
('24.149381', '120.647043')
台中市 南屯區 精科路112號
('24.14873859', '120.60823055')
台中市 南區 興大路145號1樓(中興大學學生餐廳)
('24.123535', '120.675965')
台中市 西區 精誠路7-2號
('24.156407', '120.656858')
台中市 南屯區 大墩路301號
('24.144059', '120.649746')
台中市 東區 十甲東路566號
('24.140763', '120.708345')
台中市 西屯區 漢口路二段110-7號
('24.164801', '120.661499')
台中市 大里區 永隆路35號
('24.104886', '120.677383')
台中市 南屯區 公益路二

,storeName,storeAdd,storeGps
0,基隆站前門市,基隆市 仁愛區 孝三路109號,"(25.131168, 121.739754)"
1,基隆信一門市,基隆市 中正區 信一路168號1樓,"(25.130381, 121.745334)"
2,大武崙門市,基隆市 安樂區 武嶺街136號,"(25.140018, 121.710334)"
3,基隆仁二門市,基隆市 仁愛區 仁二路66號,"(25.127855, 121.749247)"
4,基隆新豐門市,基隆市 中正區 新豐街233號,"(25.136753, 121.78828)"
...,...,...,...
500,台東中華門市,台東縣 台東市 中華路一段350號,"(22.7547708, 121.1534902)"
501,花蓮民國門市,花蓮縣 花蓮市 民國路79-3號,"(23.97769026, 121.60360466)"
502,花蓮中山門市,花蓮縣 花蓮市 中山路74號,"(23.97491593, 121.61119865)"
503,金門金城門市,金門縣 金城鎮 環島北路一段19號,"(24.44112, 118.322232)"


In [15]:
#將GPS資料拆成兩個欄位，存成csv檔後要再讀取做成地圖比較方便
data=final['storeGps'].apply(pd.Series)
data=data.rename(columns={0:'rel-store-lat',1:'rel-store-lng'})
data['storeName']=final['storeName']

#合併表格
final2 = pd.merge(final, data)
final2 = final2.drop(['storeGps'], axis=1)

In [28]:
final2

,storeName,storeAdd,rel-store-lat,rel-store-lng
0,基隆站前門市,基隆市 仁愛區 孝三路109號,25.131168,121.739754
1,基隆信一門市,基隆市 中正區 信一路168號1樓,25.130381,121.745334
2,大武崙門市,基隆市 安樂區 武嶺街136號,25.140018,121.710334
3,基隆仁二門市,基隆市 仁愛區 仁二路66號,25.127855,121.749247
4,基隆新豐門市,基隆市 中正區 新豐街233號,25.136753,121.78828
...,...,...,...,...
500,台東中華門市,台東縣 台東市 中華路一段350號,22.7547708,121.1534902
501,花蓮民國門市,花蓮縣 花蓮市 民國路79-3號,23.97769026,121.60360466
502,花蓮中山門市,花蓮縣 花蓮市 中山路74號,23.97491593,121.61119865
503,金門金城門市,金門縣 金城鎮 環島北路一段19號,24.44112,118.322232


In [37]:
final2.to_csv('louisalist.csv', mode='a+',encoding='utf-8_sig',header=True)

In [36]:
import folium
import pandas as pd
import csv

# 中文店名有亂碼問題，在這裡進行轉碼
def parse_zhch(s):
    return str(str(s).encode('ascii' , 'xmlcharrefreplace'))[2:-1]

# Louisa
m = folium.Map((23.973875, 120.982024),zoom_start=7)
i = -1
while i <=503:
    i = i + 1
    mrt = (final2.iloc[i,2],final2.iloc[i,3])
    name = final2.iloc[i,0]
    folium.Circle(
        location=mrt,
        radius=100,
        popup=parse_zhch(name),
        color='red',
        fill=False,
        fill_color='red',
        fill_opacity=0.05
    ).add_to(m)
m